In [8]:
from datasets import DatasetDict
ds= DatasetDict.from_csv("data/r_depression_posts.csv")
ds =ds.remove_columns("Unnamed: 0")
ds

Dataset({
    features: ['text', 'url', 'positive', 'negative'],
    num_rows: 32165
})

In [9]:
train_val, test = ds.train_test_split(test_size=0.03, seed=42).values()

val_size = 0.17 / (1 - 0.03)  
train, val = train_val.train_test_split(test_size=val_size, seed=42).values()

ds_splits = DatasetDict({
    'train': train,
    'validation': val,
    'test': test
})
ds_splits

DatasetDict({
    train: Dataset({
        features: ['text', 'url', 'positive', 'negative'],
        num_rows: 25731
    })
    validation: Dataset({
        features: ['text', 'url', 'positive', 'negative'],
        num_rows: 5469
    })
    test: Dataset({
        features: ['text', 'url', 'positive', 'negative'],
        num_rows: 965
    })
})